In [ ]:
# -*- coding: utf-8 -*-

"""Text Corruption Methods."""

import logging
import random

import nltk
from styleformer import Styleformer
# initialize
form_to_cas = Styleformer(style=1)
cas_to_form = Styleformer(style=0)
pass_to_act = Styleformer(style=3)
act_to_pass = Styleformer(style=2)
back_trans = BackTrans(
        from_model_name="facebook/wmt19-en-de",
        to_model_name="facebook/wmt19-de-en",
        device=None,
    )


In [4]:
import json

with open('/mnt/petrelfs/shizhelun/wangzp/data/dataset/new_robust/MMBench_C/MMBench_C.json', 'rb') as f:
    data_c = json.load(f)
with open('/mnt/petrelfs/shizhelun/wangzp/code/ChEF/results/LLaVA/Calibration/MMBench/20231017191836/MMBench_20231017195849.json', 'rb') as f:
    data = json.load(f)

res={}
total =0
cnt=0
hh=0
for a,b in zip(data,data_c):
    main_id = int(a['id'])%1e6
    if main_id in res:
        continue
    res[main_id]=1
    total +=1
    if a['gt_choices'][a['gt_choice']]!=b['gt_choices'][b['gt_choice']]:
        cnt+=1
    if a['gt_choices'][a['gt_choice']]==b['gt_choices'][((b['gt_choice']-2)+len(b['gt_choices']))%len(b['gt_choices'])]:
        hh+=1
    
print(cnt/total)
total,hh,cnt


0.0


(1176, 91, 0)

In [5]:
import json

with open('/mnt/petrelfs/shizhelun/wangzp/data/dataset/new_robust/ScienceQA_C/VQA_ScienceQA_C.json', 'rb') as f:
    data_c = json.load(f)
with open('/mnt/petrelfs/shizhelun/shizhelun/data/datasets/LAMM/2D_Benchmark/meta_file/VQA_ScienceQA.json', 'rb') as f:
    data = json.load(f)

res={}
total =0
cnt=0
hh=0
print(data[0])
print(data_c[0])
for a,b in zip(data,data_c):
    main_id = int(a['id'])%1e6
    if main_id in res:
        continue
    res[main_id]=1
    total +=1
    if a['gt_choices'][a['gt_choice']]!=b['gt_choices'][b['gt_choice']]:
        cnt+=1
    if a['gt_choices'][a['gt_choice']]==b['gt_choices'][((b['gt_choice']-2)+len(b['gt_choices']))%len(b['gt_choices'])]:
        hh+=1
    
print(cnt/total)
total,hh,cnt

{'query': "Which of the following could Gordon's test show?\nContext: People can use the engineering-design process to develop solutions to problems. One step in the process is testing if a potential solution meets the requirements of the design.\nThe passage below describes how the engineering-design process was used to test a solution to a problem. Read the passage. Then answer the question below.\n\nGordon was an aerospace engineer who was developing a parachute for a spacecraft that would land on Mars. He needed to add a vent at the center of the parachute so the spacecraft would land smoothly. However, the spacecraft would have to travel at a high speed before landing. If the vent was too big or too small, the parachute might swing wildly at this speed. The movement could damage the spacecraft.\nSo, to help decide how big the vent should be, Gordon put a parachute with a 1 m vent in a wind tunnel. The wind tunnel made it seem like the parachute was moving at 200 km per hour. He ob

(2017, 677, 0)

In [3]:
from text_corruption import d_basic,d_sentence, d_word, d_character, d_option
import torch
import numpy as np
import random

seed_value = 2024  # 设置种子值


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def clean_question(question, generative = False): # delete context
    qlist = question.split('Options:')
    q = qlist[0].split('Context:')
    if not generative:
        res = 'Question: ' + q[0] + 'Options:' + qlist[1] + "\n"
    else:
        res = 'Question: ' + q[0] + "\n"
    
    return res
sample = data[0]
text = sample['query']
qlist = text.split('Options:')
q = qlist[0].split('Context:')

context = q[1]
print(context)
text = q[0]
for k,v in d_basic.items():
    text = v(text)
    print(f'{k}: {text}')
for k,v in d_sentence.items():
    text = v(text)
    print(f'{k}: {text}')
for k,v in d_word.items():
    qtext = v(text)
    print(f'{k}: {qtext}')
for k,v in d_character.items():
    qtext = v(text)
    print(f'{k}: {qtext}')

/mnt/petrelfs/shizhelun/miniconda3/envs/corruption/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/mnt/petrelfs/shizhelun/miniconda3/envs/corruption/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: be268621-22f7-4086-a76d-4b3a4797fe58)')' thrown while requesting HEAD https://huggingface.co/prithivida/parrot_adequacy_model/resolve/main/config.json


Formal to Casual model loaded...
Casual to Formal model loaded...
Passive to Active model loaded...
Active to Passive model loaded...
 People can use the engineering-design process to develop solutions to problems. One step in the process is testing if a potential solution meets the requirements of the design.
The passage below describes how the engineering-design process was used to test a solution to a problem. Read the passage. Then answer the question below.

Gordon was an aerospace engineer who was developing a parachute for a spacecraft that would land on Mars. He needed to add a vent at the center of the parachute so the spacecraft would land smoothly. However, the spacecraft would have to travel at a high speed before landing. If the vent was too big or too small, the parachute might swing wildly at this speed. The movement could damage the spacecraft.
So, to help decide how big the vent should be, Gordon put a parachute with a 1 m vent in a wind tunnel. The wind tunnel made it

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


back_trans: Which of them could Gordon screw up the Test show?
swap_syn: Which of them could Gordon screw up the Test appearance?
insert_adv: Which of them could Gordon slowly screw up the Test show?
append_irr_corruption: In addition, Which of them could Gordon screw up the Test show? Money answers all things.
char_ocr: Which of them could Gordon sckew up the Test show?
char_typos: Which of them could Gordon screw up the eTst show?
char_spell_error: Which of them could Gordo screw up the Test show?
char_keyboard_corruption: WJich of them could Gordon screw up the Test show?


In [7]:
from text_corruption import d_basic,d_sentence, d_word, d_character, d_option
for k,v in d_option.items():
    qtext, gt_choices, gt_choice= v(text,sample['gt_choices'],sample['gt_choice'],context)
    print(f'{k}: {qtext}')
    print(gt_choices)
    print(gt_choice)
print(data[0]['gt_choices'][sample['gt_choice']])

circular_option: Question: Which of them could Gordon screw up the Test show?
Context: People can use the engineering-design process to develop solutions to problems. One step in the process is testing if a potential solution meets the requirements of the design.
The passage below describes how the engineering-design process was used to test a solution to a problem. Read the passage. Then answer the question below.

Gordon was an aerospace engineer who was developing a parachute for a spacecraft that would land on Mars. He needed to add a vent at the center of the parachute so the spacecraft would land smoothly. However, the spacecraft would have to travel at a high speed before landing. If the vent was too big or too small, the parachute might swing wildly at this speed. The movement could damage the spacecraft.
So, to help decide how big the vent should be, Gordon put a parachute with a 1 m vent in a wind tunnel. The wind tunnel made it seem like the parachute was moving at 200 km pe

In [3]:
import json
with open('/mnt/petrelfs/shizhelun/wangzp/code/ChEF/data/datasets/ChEF/MMBench_C/MMBench_C.json','rb') as f:
    data = json.load(f)
for dt in data:
    dt['query'].replace(' There are several options:', '\n There are several options:')
print(data[0]['query'])


 Fr instancew, the passaeg below describes an experiment. rea the psasage and thqn follow the instructions below. maedlyn applied a tihn layer of wax ot the underside of her sonwboard and roed the bard straight sdown a hill. then, she rmeoved the wax and roed the snowboard straight dwon the hixll agani. she repeated the rides our myre timse, alternkting whhether she rode with a thn layer of awx on the bdoard or ndt. her firend tucker tmied each riqde. madelyn and tuvcker calculated the averge tiwme it took t tslide tsraight odwn the hill on the snowboard with wa cmopared o the average time. fqigure: snowboarding donw a nill. idntify the quesption that madelyn and tfucker'sa xperiment cap bets ansjwer. Fortune fakvors the bold. 
There are several options: (A) Does Madelyn's snowboard slide down a hill in less time when it has a layer of wax or when it does not have a layer of wax? (B) Does Madelyn's snowboard slide down a hill in less time when it has a thin layer of wax or a thick laye